# 提示词(prompt)基本原则
## 1. 编写清楚、具体的指令

In [6]:
# 功能代码，服务于下面的代码片段
import os
from openai import OpenAI

def get_completion(query_txt, temperature=0,is_stream=True):
    client = OpenAI(api_key='sk-8a2970d003e849e1a680a377eb16d22b',
                base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {"role": "user", "content": query_txt},
        ],
        temperature = temperature,
        stream=is_stream,
        # stream_options={"include_usage": True if is_stream else False}
    )

    if not is_stream:
        # print(completion.model_dump_json())
        print(completion.choices[0].message.content)
    else:
        for chunk in completion:
            # print(chunk.model_dump_json())
            if len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if delta and delta.content:
                    # print(f"id: {chunk.id}, content: {chunk.choices[0].delta.content}")
                    print(chunk.choices[0].delta.content,end="")


1.1 使用分隔符清晰的表示输入的不同部分

分隔符就像是 Prompt 中的墙，将不同的指令、上下文、输入隔开，避免意外的混淆。你可以选择用```，"""，< >，<tag> </tag>，: 等做分隔符，只要能明确起到隔断作用即可。

In [7]:
#测试提示词中的间隔符
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句详细话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
get_completion(prompt)


为了获得更详细和相关的输出，您应提供清晰、具体的指示来表达任务需求，避免混淆清晰与简短，因为在许多情况下，较长的提示词可通过提供更多上下文信息来提高模型输出的准确性和相关性。

1.2 要求结构化的输出

有时候我们需要语言模型给我们一些结构化的输出，而不仅仅是连续的文本。
什么是结构化输出呢？就是按照某种格式组织的内容，例如JSON、HTML等。这种输出非常适合在代码
中进一步解析和处理。例如，您可以在 Python 中将其读入字典或列表中。

In [ ]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构的、非真实存在的中文书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)


"""
[
    {
        "book_id": 1,
        "title": "星河彼端的秘密",
        "author": "林宇辰",
        "genre": "科幻"
    },
    {
        "book_id": 2,
        "title": "风中消失的古城",
        "author": "沈婉清",
        "genre": "奇幻"
    },
    {
        "book_id": 3,
        "title": "时光深处的低语",
        "author": "叶明浩",
        "genre": "悬疑"
    }
]
"""

1.3 要求模型检查是否满足条件

如果任务包含不一定能满足的假设（条件），我们可以告诉模型先检查这些假设，如果不满足，则会指
出并停止执行后续的完整流程。您还可以考虑可能出现的边缘情况及模型的应对，以避免意外的结果或
错误发生。

In [ ]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""

text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""

prompt1 = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
… 第
N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
print("测试一： 满足条件的")
response = get_completion(prompt1)

prompt2 = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
… 第
N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
print("\n测试二： 不满足条件的")
response = get_completion(prompt2)

1.4 提供少量示例

要求模型执行实际任务之前，给模型一两个已完成的样例，让模型了解我们的要求和期望的输出样式。类似给了模型一个限定和模仿的样式

In [ ]:
prompt = f"""
您的任务是以一致的风格回答问题，完成两轮会话。
<孩子>: 请教我何为耐心。
<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以
一根孤独的线开始编织。
<孩子>: 请教我何为韧性。
"""
response = get_completion(prompt)

## 2. 给模型时间思考

在设计 Prompt 时，给予语（分步骤进行分析处理）。语言模型与人类一样，需要时间来思考并
解决复杂问题。如果让语言模型匆忙给出结论，其结果很可能不准确。例如，若要语言模型推断一本书
的主题，仅提供简单的书名和一句简介是不足够的。这就像让一个人在极短时间内解决困难的数学题，
错误在所难免。
相反，我们应通过 Prompt 指引语言模型进行深入思考。可以要求其先列出对问题的各种看法，说明推
理依据，然后再得出最终结论。在 Prompt 中添加逐步推理的要求，能让语言模型投入更多时间逻辑思
维，输出结果也将更可靠准确。

2.1 指定完成任务所需的步骤


In [ ]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：english_summary，num_names。
请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text:<{text}>
"""
response = get_completion(prompt_1)

2.2 指导模型在下结论之前找出一个自己的想法

明确指导语言模型进行自主思考

In [ ]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：
步骤：
首先，自己解决问题。
然后将您的解决方案与学生的解决方案进行比较，对比计算得到的总费用与学生计算的总费用是否一致，
并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。
使用以下格式：
问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
学生计算的总费用：学生计算得到的总费用
实际计算的总费用：实际计算出的总费用
学生计算的费用和实际计算的费用是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否
学生的成绩：正确或不正确
问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x美元
2. 太阳能电池板费用：250x美元
3. 维护费用：100,000+100x=10万美元+10x美元
总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元
实际解决方案和步骤：
"""
response = get_completion(prompt)

## 3. 使用提示词对抗大模型的幻觉

先让语言模型直接引用文本中的原句，然后再进行解
答。这可以追踪信息来源，降低虚假内容的风险。

In [ ]:
prompt = f"""
告诉我华为公司生产的GT Watch运动手表的相关信息。
要求写出具体的引用原文，如果有官网信息，一并提供
"""
response = get_completion(prompt)

## 4. 提示词的的迭代优化

有了任务想法后，可以先编写初版 Prompt，注意清晰明确并给模型充足思考时间。运行后检查
结果，如果不理想，则分析 Prompt 不够清楚或思考时间不够等原因，做出改进，再次运行。如此循环
多次，终将找到适合应用的 Prompt。


In [ ]:
# 初始提示词

fact_sheet_chair = """
概述
美丽的中世纪风格办公家具系列的一部分，包括文件柜、办公桌、书柜、会议桌等。
多种外壳颜色和底座涂层可选。
可选塑料前后靠背装饰（SWC-100）或10种面料和6种皮革的全面装饰（SWC-110）。
底座涂层选项为：不锈钢、哑光黑色、光泽白色或铬。
椅子可带或不带扶手。
适用于家庭或商业场所。
符合合同使用资格。
结构
五个轮子的塑料涂层铝底座。
气动椅子调节，方便升降。
尺寸
宽度53厘米|20.87英寸
深度51厘米|20.08英寸
高度80厘米|31.50英寸
座椅高度44厘米|17.32英寸
座椅深度41厘米|16.14英寸
选项
软地板或硬地板滚轮选项。
两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。
无扶手或8个位置PU扶手。
材料
外壳底座滑动件
改性尼龙PA6/PA66涂层的铸铝。
外壳厚度：10毫米。
座椅
HD36泡沫
原产国
意大利
"""


prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的营销描述。
根据```标记的技术说明书中提供的信息，编写一个产品描述。
技术说明: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

In [ ]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。
根据```标记的技术说明书中提供的信息，编写一个产品描述。
使用最多50个词。
技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)

In [ ]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。
根据```标记的技术说明书中提供的信息，编写一个产品描述。
该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。
使用最多50个单词。
技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

In [ ]:
# 更进一步，增加产品ID信息
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。
根据```标记的技术说明书中提供的信息，编写一个产品描述。
该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。
在描述末尾，包括技术规格中每个7个字符的产品ID。
使用最多50个单词。
技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

In [ ]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。
根据```标记的技术说明书中提供的信息，编写一个产品描述。
该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。
在描述末尾，包括技术规格中每个7个字符的产品ID。
在描述之后，包括一个表格，提供产品的尺寸。表格应该有两列。第一列包括尺寸的名称。第二列只包括英寸的
测量值。
给表格命名为“产品尺寸”。
将所有内容格式化为可用于网站的HTML格式。将描述放在<div>元素中。
技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)

In [ ]:
#多文本总结：

review_1  = """
这个熊猫公仔是我给女儿的生日礼物，她很喜欢，去哪都带着。
公仔很软，超级可爱，面部表情也很和善。但是相比于价钱来说，
它有点小，我感觉在别的地方用同样的价钱能买到更大的。
快递比预期提前了一天到货，所以在送给女儿之前，我自己玩了会。
"""

# 一盏落地灯的评论
review_2 = """
我需要一盏漂亮的卧室灯，这款灯不仅具备额外的储物功能，价格也并不算太高。
收货速度非常快，仅用了两天的时间就送到了。
不过，在运输过程中，灯的拉线出了问题，幸好，公司很乐意寄送了一根全新的灯线。
新的灯线也很快就送到手了，只用了几天的时间。
装配非常容易。然而，之后我发现有一个零件丢失了，于是我联系了客服，他们迅速地给我寄来了缺失的零件！
对我来说，这是一家非常关心客户和产品的优秀公司。
"""
# 一把电动牙刷的评论
review_3 = """
我的牙科卫生员推荐了电动牙刷，所以我就买了这款。
到目前为止，电池续航表现相当不错。
初次充电后，我在第一周一直将充电器插着，为的是对电池进行条件养护。
过去的3周里，我每天早晚都使用它刷牙，但电池依然维持着原来的充电状态。
不过，牙刷头太小了。我见过比这个牙刷头还大的婴儿牙刷。
我希望牙刷头更大一些，带有不同长度的刷毛，
这样可以更好地清洁牙齿间的空隙，但这款牙刷做不到。
总的来说，如果你能以50美元左右的价格购买到这款牙刷，那是一个不错的交易。
制造商的替换刷头相当昂贵，但你可以购买价格更为合理的通用刷头。
这款牙刷让我感觉就像每天都去了一次牙医，我的牙齿感觉非常干净！
"""
# 一台搅拌机的评论
review_4 = """
在11月份期间，这个17件套装还在季节性促销中，售价约为49美元，打了五折左右。
可是由于某种原因（我们可以称之为价格上涨），到了12月的第二周，所有的价格都上涨了，
同样的套装价格涨到了70-89美元不等。而11件套装的价格也从之前的29美元上涨了约10美元。
看起来还算不错，但是如果你仔细看底座，刀片锁定的部分看起来没有前几年版本的那么漂亮。
然而，我打算非常小心地使用它
（例如，我会先在搅拌机中研磨豆类、冰块、大米等坚硬的食物，然后再将它们研磨成所需的粒度，
接着切换到打蛋器刀片以获得更细的面粉，如果我需要制作更细腻/少果肉的食物）。
在制作冰沙时，我会将要使用的水果和蔬菜切成细小块并冷冻
（如果使用菠菜，我会先轻微煮熟菠菜，然后冷冻，直到使用时准备食用。
如果要制作冰糕，我会使用一个小到中号的食物加工器），这样你就可以避免添加过多的冰块。
大约一年后，电机开始发出奇怪的声音。我打电话给客户服务，但保修期已经过期了，
所以我只好购买了另一台。值得注意的是，这类产品的整体质量在过去几年里有所下降
，所以他们在一定程度上依靠品牌认知和消费者忠诚来维持销售。在大约两天内，我收到了新的搅拌机。
"""

reviews = [review_1, review_2, review_3, review_4]
for i in range(len(reviews)):
    prompt = f"""
        你的任务是从电子商务网站上的产品评论中提取相关信息。
        请对三个反引号之间的评论文本进行概括，最多20个词汇。
        评论文本: ```{reviews[i]}```
        """
    print(f"第 {i+1} 评论:")
    response = get_completion(prompt)
    print()

## 5. 利用提示词让大模型推理

5.1 情感推理

- 情感倾向分析
- 识别情感类型
- 识别愤怒

In [ ]:
lamp_review = """
我需要一盏漂亮的卧室灯，这款灯具有额外的储物功能，价格也不算太高。\
我很快就收到了它。在运输过程中，我们的灯绳断了，但是公司很乐意寄送了一个新的。\
几天后就收到了。这款灯很容易组装。我发现少了一个零件，于是联系了他们的客服，他们很快就给我寄来了缺
失的零件！\
在我看来，Lumina 是一家非常关心顾客和产品的优秀公司！
"""
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？
评论文本: ```{lamp_review}```
"""
response = get_completion(prompt)

print()
prompt2 = f"""
识别以下评论的作者表达的情感。包含不超过五个项目。将答案格式化为以逗号分隔的单词列表。
评论文本: ```{lamp_review}```
"""
get_completion(prompt2)

In [ ]:
#通过提示词，格式化输出提取的信息：

prompt = f"""
从评论文本中识别以下项目：
- 情绪（正面或负面）
- 审稿人是否表达了愤怒？（是或否）
- 评论者购买的物品
- 制造该物品的公司
评论用三个反引号分隔。将你的响应格式化为 JSON 对象，以 “情感倾向”、“是否生气”、“物品类型” 和
“品牌” 作为键。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
将 “是否生气” 值格式化为布尔值。
评论文本: ```{lamp_review}```
"""
response = get_completion(prompt)

5.2 主题推断

In [ ]:
story = """
在政府最近进行的一项调查中，要求公共部门的员工对他们所在部门的满意度进行评分。
调查结果显示，NASA 是最受欢迎的部门，满意度为 95％。
一位 NASA 员工 John Smith 对这一发现发表了评论，他表示：
“我对 NASA 排名第一并不感到惊讶。这是一个与了不起的人们和令人难以置信的机会共事的好地方。我为成
为这样一个创新组织的一员感到自豪。”
NASA 的管理团队也对这一结果表示欢迎，主管 Tom Johnson 表示：
“我们很高兴听到我们的员工对 NASA 的工作感到满意。
我们拥有一支才华横溢、忠诚敬业的团队，他们为实现我们的目标不懈努力，看到他们的辛勤工作得到回报是太
棒了。”
调查还显示，社会保障管理局的满意度最低，只有 45％的员工表示他们对工作满意。
政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，
以python列表的形式给出答案，要求每个元素是一个Json对象，键为对应主题，值为对应的 0 或 1。
主题列表：美国航空航天局、当地政府、工程、员工满意度、联邦政府
给定文本: ```{story}```
"""
response = get_completion(prompt)
print("----------------------")
print(response)


## 6 文本转化

大语言模型具有强大的文本转换能力，可以实现多语言翻译、拼写纠正、语法调整、格式转换等不同类
型的文本转换任务。利用语言模型进行各类转换是它的典型应用之一。

6.1 文本翻译

In [ ]:
print("西班牙语:")
prompt = f"""
将以下中文翻译成西班牙语: \
```您好，我想订购一个搅拌机。```
"""
response = get_completion(prompt)

print("\n英语:")
prompt2 = f"""
将以下中文翻译成英语: \
```您好，我想订购一个搅拌机。```
"""
get_completion(prompt2)

prompt3 = f"""
将以下中文翻译成日语: \
```您好，我想订购一个搅拌机。```
"""
print("\n日语:")
get_completion(prompt3)

print("")
prompt4 = f"""
请将以下文本分别翻译成中文、英文、法语和西班牙语:
```I want to order a basketball.```
"""
response = get_completion(prompt4)

6.2 其他转化
- 文件格式转化
- 拼写及语法纠正
- 语气与写作风格调整

In [ ]:
data_json = { "resturant employees" :[
{"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
{"name":"Bob", "email":"bob32@gmail.com"},
{"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
将以下Python字典从JSON转换为HTML表格，保留表格标题和列名：{data_json}
"""
response = get_completion(prompt)

print()
prompt2 = f"""
将以下Python字典从JSON转换为markdown格式，保留表格标题和列名：{data_json}
"""
response = get_completion(prompt2)


In [ ]:
#综合示例：

text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room. Yes, adults also like pandas too. She takes \
it everywhere with her, and it's super soft and cute. One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price. It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""

prompt = f"""
针对以下三个反引号之间的英文评论文本，
首先进行拼写及语法纠错，
然后将其转化成中文，
再将其转化成优质淘宝评论的风格，从各种角度出发，分别说明产品的优点与缺点，并进行总结。
润色一下描述，使评论更具有吸引力。
输出结果格式为：
【优点】xxx
【缺点】xxx
【总结】xxx
注意，只需填写xxx部分，并分段输出。
将结果输出成Markdown格式。
```{text}```
"""
response = get_completion(prompt)

## 7. 文本扩展

文本扩展是大语言模型的一个重要应用方向，它可以输入简短文本，生成更加丰富的长文。这为创作提
供了强大支持，但也可能被滥用。

In [103]:
# 邮件回复

# 我们可以在推理那章学习到如何对一个评论判断其情感倾向
sentiment = "消极的"
# 一个产品的评价
review = f"""
他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。\
但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。\
11件套装的价格也上涨了大约10美元左右。\
虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。\
不过我打算非常温柔地使用它，例如，\
我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，\
切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效
果。\
制作冰沙时，特别提示：\
将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；\
如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。\
大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。\
总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。\
货物在两天内到达。
"""

prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。
根据客户通过“```”分隔的评价，生成回复以感谢客户的评价。提醒模型使用评价中的具体细节
用简明而专业的语气写信。
作为“AI客户代理”签署电子邮件。
客户评论：
```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt,is_stream=False)

主题：感谢您的反馈与建议  

尊敬的客户，  

感谢您抽出时间分享对我们的产品和服务的详细评价。我们非常重视您的意见，并认真对待您提到的问题。  

您提到11月份季节性销售期间的价格优惠与后续价格波动的情况，我们理解这可能给您带来了困惑和不便。我们将进一步审查定价策略，确保为客户提供公平且透明的价值体验。  

关于您对产品设计和质量的观察，特别是基座锁定部分的变化以及电机在使用一段时间后出现的噪音问题，我们深表关注。虽然您提到保修已过期，但我们始终致力于通过改进产品设计和增强耐用性来提升客户的长期满意度。您的温柔使用方式和实用技巧也为我们提供了宝贵的参考，我们会将这些信息传递给研发团队。  

最后，尽管您对产品质量下降表示了担忧，但对我们快速配送服务的认可让我们倍感欣慰。我们将继续努力，在各个方面提供更好的体验以回报客户的信任和支持。  

再次感谢您的坦诚反馈。如果您有任何其他问题或需要进一步的帮助，请随时联系我们。  

此致  
AI客户代理


In [ ]:
# 温度系数
# 大语言模型中的 “温度”(temperature) 参数可以控制生成文本的随机性和多样性。temperature 的值越
# 大，语言模型输出的多样性越大；temperature 的值越小，输出越倾向高概率的文本。

get_completion(prompt, temperature=0.1)
get_completion(prompt, temperature=0.7)

# 温度（temperature）参数可以控制语言模型生成文本的随机性。温度为0时，每次使用同样的
# Prompt，得到的结果总是一致的。而在上面的样例中，当温度设为0.7时，则每次执行都会生成不同的文本。

## 8. 聊天机器人

8.1 给定身份

传入一个消息列表，设定消息中各种不同的角色 (roles) ，我们会描述一下这些角色

messages = [
{'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},   ---> 系统角色，它提供了一个总体的指示。系统消息则有助于设置助手的行为和角色，并作为对话的高级指示。你可以想象它在助手的耳边低语，引导它的回应，而用户不会注意到系统消息。
{'role':'user', 'content':'给我讲个笑话'},    ---> 用户角色，它表示用户所期待的助手的回应。
{'role':'assistant', 'content':'鸡为什么过马路'},   ---> 助手角色，它表示助手所期待的回应。
{'role':'user', 'content':'我不知道'} ]

In [8]:
import inspect
def get_completion_from_messages(messages, model="qwen-max", temperature=0):
    client = OpenAI(api_key='sk-8a2970d003e849e1a680a377eb16d22b',
                base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
    completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, # 控制模型输出的随机程度
    )
    # print(str(response.choices[0].message))
    token_dict = {
        'prompt_tokens':completion.usage.prompt_tokens,
        'completion_tokens':completion.usage.completion_tokens,
        'total_tokens':completion.usage.total_tokens,
    }
    print("token_dict: %s" % token_dict)
    # print(inspect.getmembers(completion))
    return completion.choices[0].message.content

In [9]:
messages = [
    {'role':'system', 'content':'你是个友好的聊天机器人。'},
    {'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'} ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

token_dict: {'prompt_tokens': 31, 'completion_tokens': 27, 'total_tokens': 58}
当然可以，不过你还没告诉我你的名字是什么呢。如果你愿意的话，可以告诉我你的名字，这样我就能帮你记住了。


In [10]:
messages = [
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'} ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

token_dict: {'prompt_tokens': 66, 'completion_tokens': 21, 'total_tokens': 87}
当然可以！你的名字是Isa。还有其他事情需要我提醒你或者帮助你的吗？


In [110]:
!pip install panel

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/29.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/29.5 MB ? eta -:--:--
    --------------------------------------- 0.5/29.5 MB 1.3 MB/s eta 0:00:23
   - -------------------------------------- 1.0/29.5 MB 1.4 MB/s eta 0:00:21
   - -------------------------------------- 1.3/29.5 MB 1.3 MB/s eta 0:00:22
   -- ------------------------------------- 1.6/29.5 MB 1.4 MB/s eta 0:00:20
   -- ------------------------------------- 2.1/29.5 MB 1.5 MB/s eta 0:00:18
   --- ------------------------------------ 2.6/29.5 MB 1.6 MB/s eta 0:00:17
   --- ------------------------------------ 2.9/29.5 MB 1.7 MB/s eta 0:00:17
   ---- ---------------------------------

In [ ]:
import panel as pn # GUI

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
    return pn.Column(*panels)


pn.extension()
panels = [] # collect display
context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。
请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。
菜单包括：
菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25
配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00
饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ] # accumulate messages
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
interactive_conversation = pn.bind(collect_messages, button_conversation)
dashboard = pn.Column(inp,pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
) 
dashboard

# 8. 思维连推理
在查询中明确要求语言模型先提供一系列相关推理步骤，进行深度思考，然后再给出最终答案，这更接近人类解题的
思维过程。
相比直接要求输出结果，这种引导语言模型逐步推理的方法，可以减少其匆忙错误，生成更准确可靠的
响应。思维链推理使语言模型更好地模拟人类逻辑思考，是提升其回答质量的重要策略之一。

## 8.1 思维链提示设计
思维连提示是一种引导语音模型进行逐步推理的Prompt设计方法。它通过在Prompt中设置系统消息，要求语言模型在给出最终结论之前，先明确各个推理步骤：
* 步骤1：先请语言模型陈述对问题的初步理解，然后列出需要考虑的方方面面
* 步骤2：再逐步分析这些原因，给出支持或反对的结论
* 步骤3：最后得出整体结论

**如下所示，系统消息的设计**


In [ ]:
delimiter = "===="
system_message = f"""
请按照以下步骤回答客户的提问。客户的提问将以{delimiter}分隔。
步骤 1:{delimiter}首先确定用户是否正在询问有关特定产品或产品的问题。产品类别不计入范围。
步骤 2:{delimiter}如果用户询问特定产品，请确认产品是否在以下列表中。所有可用产品：

产品：TechPro 超极本
类别：计算机和笔记本电脑
品牌：TechPro
型号：TP-UB100
保修期：1 年
评分：4.5
特点：13.3 英寸显示屏，8GB RAM，256GB SSD，Intel Core i5 处理器
描述：一款适用于日常使用的时尚轻便的超极本。
价格：$799.99

产品：BlueWave 游戏笔记本电脑
类别：计算机和笔记本电脑
品牌：BlueWave
型号：BW-GL200
保修期：2 年
评分：4.7
特点：15.6 英寸显示屏，16GB RAM，512GB SSD，NVIDIA GeForce RTX 3060
描述：一款高性能的游戏笔记本电脑，提供沉浸式体验。
价格：$1199.99

产品：PowerLite 可转换笔记本电脑
类别：计算机和笔记本电脑
品牌：PowerLite
型号：PL-CV300
保修期：1年
评分：4.3
特点：14 英寸触摸屏，8GB RAM，256GB SSD，360 度铰链
描述：一款多功能可转换笔记本电脑，具有响应触摸屏。
价格：$699.99

产品：TechPro 台式电脑
类别：计算机和笔记本电脑
品牌：TechPro
型号：TP-DT500
保修期：1年
评分：4.4
特点：Intel Core i7 处理器，16GB RAM，1TB HDD，NVIDIA GeForce GTX 1660
描述：一款功能强大的台式电脑，适用于工作和娱乐。
价格：$999.99

产品：BlueWave Chromebook
类别：计算机和笔记本电脑
品牌：BlueWave
型号：BW-CB100
保修期：1 年
评分：4.1
特点：11.6 英寸显示屏，4GB RAM，32GB eMMC，Chrome OS
描述：一款紧凑而价格实惠的 Chromebook，适用于日常任务。
价格：$249.99

步骤 3:{delimiter} 如果消息中包含上述列表中的产品，请列出用户在消息中做出的任何假设，\
例如笔记本电脑 X 比笔记本电脑 Y 大，或者笔记本电脑 Z 有 2 年保修期。
步骤 4:{delimiter} 如果用户做出了任何假设，请根据产品信息确定假设是否正确。
步骤 5:{delimiter} 如果用户有任何错误的假设，请先礼貌地纠正客户的错误假设（如果适用）。\
只提及或引用可用产品列表中的产品，因为这是商店销售的唯一五款产品。以友好的口吻回答客户。
使用以下格式回答问题：
步骤 1: {delimiter} <步骤 1 的推理>
步骤 2: {delimiter} <步骤 2 的推理>
步骤 3: {delimiter} <步骤 3 的推理>
步骤 4: {delimiter} <步骤 4 的推理>
回复客户: {delimiter} <回复客户的内容>
请确保每个步骤上面的回答中都使用 {delimiter} 对步骤和步骤的推理进行分隔。
"""

user_message = f"""BlueWave Chromebook 比 TechPro 台式电脑贵多少？"""
messages = [
	{'role':'system','content': system_message},
	{'role':'user','content': f"{delimiter}{user_message}{delimiter}"},
] 
response = get_completion_from_messages(messages)
print(response)

In [17]:
user_message = f"""你有电视机么"""
messages = [
	{'role':'system','content': system_message},
	{'role':'user','content': f"{delimiter}{user_message}{delimiter}"},
] 
response = get_completion_from_messages(messages)
print(response)


token_dict: {'prompt_tokens': 764, 'completion_tokens': 124, 'total_tokens': 888}
步骤 1: ==== 用户没有询问特定产品或产品类别，而是在询问是否有电视机。
步骤 2: ==== 由于用户提到的产品（电视机）不在提供的产品列表中，因此无需进一步检查具体型号。
步骤 3: ==== 用户没有做出任何关于现有产品列表中的假设。
步骤 4: ==== 不适用，因为用户的询问与当前可售产品无关。
回复客户: ==== 您好！感谢您的询问。目前我们这里主要销售的是笔记本电脑和台式机等计算机类产品，并不提供电视机哦。如果您对我们的其他电子产品感兴趣的话，请随时告诉我！


## 8.2 内心独白
通过分析大模型的回复根据一些特定约束字符，隐藏推理链，或者保护敏感信息。

In [21]:
try:
	if delimiter in response:
		final_response = response.split(delimiter)[-1].strip()
		# tmp = response.split(delimiter)[-2].strip()
		# print(tmp)
	else:
		final_response = response.split(":")[-1].strip()
except Exception as e:
	final_response = "对不起，我现在有点问题，请尝试问另外一个问题"
print(final_response)

您好！感谢您的询问。目前我们这里主要销售的是笔记本电脑和台式机等计算机类产品，并不提供电视机哦。如果您对我们的其他电子产品感兴趣的话，请随时告诉我！


# 9 处理输入--链式
链式提示是将复杂任务分解为多个简单Prompt的策略。它的优点如下：
* 分解复杂度，每个Pormpt只处理一个具体子任务，避免过于宽泛的要求，提高成功率
* 降低计算成本，过程的Prompt使用更大tokes，增加计算成本
* 更容易测试和调试
* 更灵活的工作流程，根据不同情况可以进行不同的操作